In [ ]:
#-------------------------STEP one : Automatically crop the volumes -----------------------------------------------
#-------------with help of segmentation and information of the lesion------------------------- 
#cropp is disabled currently
def preprocess(seg_name, data_name, array_verte, store_name, folder = 'networks', end_size = 200, plot = False, treshold = -1):
    
    ##############loading#################
    DS = False
    seg = load_data(seg_name)
    data, img = load_data2(data_name)
    print('shape data :'+ str(data.shape))
    print('shape seg :'+ str(seg.shape))
    
    ############cropping#########################
    cropped_seg, cropped_data = crop_points(seg, array_verte, data) # change
    shape = cropped_data.shape
    shape_seg = cropped_seg.shape
    print('shape of the data after cropping '+ str(shape) )
    print('shape of the seg after cropping '+ str(shape_seg) )
    
    ###############plotting######################
    if (plot==True):
        #plot_middle_slices(cropped_data,treshold = -1 )
        plot_middle_slices(data,treshold = -1)
   
    
    ######downsampling###########
    if (DS == True) : 
        while (max(shape)>end_size):
            value = max(shape)
            coefficient = end_size/value
            if (coefficient< 0.93):
                print('yes we are sampling down with coefficent: '+ str(coefficient))
                cropped_data = downsample(cropped_data)
                cropped_seg = downsample(cropped_seg)
            else:
                arg_max = np.argmax(shape)
                diff = int((value-end_size)/2)
                if (arg_max == 0):
                    cropped_data = cropped_data[diff:(end_size-diff),:,:]
                    cropped_seg = cropped_seg[diff:(end_size-diff),:,:]
                elif (arg_max == 1): 
                    cropped_data = cropped_data[:,diff:(end_size-diff),:]
                    cropped_seg = cropped_seg[:,diff:(end_size-diff),:]
                elif (arg_max == 2):
                    cropped_data = cropped_data[:,:,diff:(end_size-diff)]
                    cropped_seg = cropped_data[:,:,diff:(end_size-diff)]


            shape = cropped_data.shape

        
    
    ######padding####################
    #pad_new_img = center_pad(cropped_data,np.array([end_size,end_size,end_size]))
    #pad_seg_img = center_pad(cropped_seg,np.array([end_size,end_size,end_size]))
    #shape = pad_new_img.shape
    #if (plot==True):
    #    plot_middle_slices(pad_seg_img)
    #    plot_middle_slices(pad_new_img)
    #print('shape of the data after padding '+ str(shape) )
    pad_new_img = cropped_data 
    pad_seg_img = cropped_seg
    #############storing################
    
    pad_nifty_img = nib.Nifti1Image(pad_new_img, affine=img.affine)
    #print(type(pad_nifty_img))
    #print(pad_nifty_img.shape)
    nib.save(pad_nifty_img,os.path.join(folder,store_name + '.nii.gz'))
    ####HERE WE COULD SAVE THE SAME JUST TO SEE##
    pad_nifty_seg = nib.Nifti1Image(pad_seg_img, affine=np.eye(4))
    #print(type(pad_nifty_img))
    #print(pad_nifty_img.shape)
    nib.save(pad_nifty_seg,os.path.join(folder,store_name + 'seg.nii.gz'))
    print('data has been stored in folder :' + str(folder))

def crop_points(arr, numbers, data = -1):
    concat_x=[]
    concat_y=[]
    concat_z=[]
    shape = arr.shape
    
    string = "Vertebras selected are:"
    for i in range(0,len(numbers)):
        #'result' % x = np.where(arr==numbers[i])
        string = string + str(numbers[i])+ " "
        globals()['result{}'.format(i)] = np.where(arr==numbers[i])
    print(string)
        
    for i in range(0,len(numbers)):
        #print('here')
        concat_x= np.concatenate((concat_x,globals()['result{}'.format(i)][0]),axis=0)
        concat_y= np.concatenate((concat_y,globals()['result{}'.format(i)][1]),axis=0)
        concat_z= np.concatenate((concat_z,globals()['result{}'.format(i)][2]),axis=0)
    #print('res0')
    #print(result0)
    max_x = int(max(concat_x))
    max_y = int(max(concat_y))
    max_z = int(max(concat_z))
    min_x = int(min(concat_x))
    min_y = int(min(concat_y))
    min_z = int(min(concat_z))
    #print(result0)
    #print(result1)
    print("x-Values from  " + str(min_x)+ " till " + str(max_x) )
    print("y-Values from  " + str(min_y)+ " /0 till " + str(max_y) )
    print("z-Values from  " + str(min_z)+ " till " + str(max_z) )
    
    ###mistake with y_min value...seems not to be right but whyy??????
    cropped_image = arr[min_x:max_x+1,min_y:max_y,min_z:max_z+1]
    if isinstance(data, int) :
        data = -1
        
    else: 
        data = data[min_x:max_x+1,min_y:max_y,min_z:max_z+1]
    return cropped_image, data          

In [ ]:
# ------------------STEP two: Downsample/crop with treshold --------------------------------------------
def downsample_by_factor(img,factor):
    down_x = img[range(0,img.shape[0],factor),:,:]
    down_y = down_x[:,range(0,img.shape[1],factor),:]
    down_z = down_y[:,:,range(0,img.shape[2],factor)]
    return down_z


#downsample in a whole folder
def downsampling_in_whole_folder(end_size, folder_data, folder_store):
    baseDir = folder_data
    files = glob(baseDir + '/*.nii')
    
    
    for file in files:
        cropped_data, img = load_data(file)
        shape = cropped_data.shape
        print(file)
        print(shape)
        if (max(shape)> end_size):
            ratio = max(shape)/end_size
            ratio_int = int(ratio)
            rest = ratio - ratio_int
            if (rest > 0.30):
                ratio_int += 1
                cropped_data = downsample_by_factor(cropped_data, ratio_int)
                
            else:
                cropped_data = downsample_by_factor(cropped_data, ratio_int)
                shape = cropped_data.shape
                while (max(shape)> end_size):
                    value = max(shape)
                    #coefficient = end_size/value
                    arg_max = np.argmax(shape)
                    #print('value')
                    #print(value)
                    #print(end_size)
                    #print((value-end_size)/2)
                    diff = int((value-end_size)/2)
                    if (arg_max == 0):
                        cropped_data = cropped_data[diff:(end_size-diff),:,:]
                    elif (arg_max == 1):
                        #print('in one')
                        #print(cropped_data.shape)
                        cropped_data = cropped_data[:,diff:(end_size-diff),:]
                        #print(diff)
                        #print(cropped_data.shape)
                    elif (arg_max == 2):
                        cropped_data = cropped_data[:,:,diff:(end_size-diff)]
                    shape = cropped_data.shape
            
        #save nifti
        print('shape: ', cropped_data.shape)
        saveDir = folder_store
        #img = nib.Nifti1Image(cropped_data, np.eye(4))
        img = nib.Nifti1Image(cropped_data, img.affine)
        nib.save(img,os.path.join(saveDir, file[len(baseDir)+1:]))

In [ ]:
#-----------------STEP three: REGISTRATION via IMFUSION -----------------------------------------------------------
# rigid registation, same spacing , reset all translation, download


In [ ]:
#-------------------------STEP four: apply the translations calculated in IMFUSION to the images -----------------
#(CAREFUL...if a translation is shown in the imfuison load that does not exist in the headerfile... then it can not
#be done automatically... extract values from hand and then apply give it to the funtion (- + is exchanged Imfusion vs nib))
def translation(data_path,translation = np.array([0,0,0]),saveDir ='.'):
    corrected_data, b = load_data(data_path)
    #corrected_data = np.array(nifty_data.dataobj)
    #translation = img.affine[:,3]
    #print('translations:',translation)
    corrected_data = np.roll(corrected_data, translation[0], axis = 0)
    corrected_data = np.roll(corrected_data, translation[1], axis = 1)
    corrected_data = np.roll(corrected_data, translation[2], axis = 2)
    corrected_nifty = nib.Nifti1Image(corrected_data, b.affine)
    nib.save(corrected_nifty, saveDir + data_path[22:] )
    return corrected_data, corrected_nifty
translation('.\\healthy_reg_applied\\healthy1_L1-L3.nii', np.array([0,0,0]), '.\\healthy_trans\\healthy1_L1-L3.nii')
translation('.\\healthy_reg_applied\\healthy2_L1-L3.nii', np.array([1,0,-1]), '.\\healthy_trans\\healthy2_L1-L3.nii')
translation('.\\healthy_reg_applied\\healthy3_L1-L3.nii', np.array([2,2,0]), '.\\healthy_trans\\healthy3_L1-L3.nii')
translation('.\\healthy_reg_applied\\healthy4_L1-L3.nii', np.array([2,1,0]), '.\\healthy_trans\\healthy4_L1-L3.nii')
translation('.\\healthy_reg_applied\\healthy5_L1-L3.nii', np.array([1,2,0]), '.\\healthy_trans\\healthy5_L1-L3.nii')
translation('.\\healthy_reg_applied\\healthy6_L1-L3.nii', np.array([0,2,0]), '.\\healthy_trans\\healthy6_L1-L3.nii')
translation('.\\healthy_reg_applied\\healthy7_L1-L3.nii', np.array([1,3,-1]), '.\\healthy_trans\\healthy7_L1-L3.nii')
translation('.\\healthy_reg_applied\\healthy8_L1-L3.nii', np.array([0,0,0]), '.\\healthy_trans\\healthy8_L1-L3.nii')
translation('.\\healthy_reg_applied\\healthy9_L1-L3.nii', np.array([0,0,1]), '.\\healthy_trans\\healthy9_L1-L3.nii')
translation('.\\healthy_reg_applied\\healthy10_L1-L3.nii', np.array([-1,-3,2]), '.\\healthy_trans\\healthy10_L1-L3.nii')

In [ ]:
#---------------STEP five: Apply img.affine to the images -------------------------------------------------------
#(it applies the rotation caclulated in IMFUISON to the images)

def round_half_up(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n*multiplier + 0.5) / multiplier
#flag scipy
def space_correction(load_dir,store_dir,plot = False, translation = False):
    nifty_vol = nib.load(load_dir)
    nifty_vol = nib.as_closest_canonical(nifty_vol)
    vol_data = np.array(nifty_vol.dataobj)
    #matrix = np.linalg.inv(img.affine)
    #img.affine[:,3] = [0,0,0,1]
    #print(img.affine)
    out_vox_map = nib.spaces.vox2out_vox((nifty_vol.shape, nifty_vol.affine))
    corrected_nifty = nib.processing.resample_from_to(nifty_vol,out_vox_map)
    
    if(translation == True):
        corrected_data = np.array(corrected_nifty.dataobj)
        translation = nifty_vol.affine[:,3]
        print('translations:',translation)
        corrected_data = np.roll(corrected_data, int(round_half_up(translation[0])) , axis = 0)
        corrected_data = np.roll(corrected_data, int(round_half_up(translation[1])), axis = 1)
        corrected_data = np.roll(corrected_data, int(round_half_up(translation[2])) , axis = 2)
        corrected_nifty = nib.Nifti1Image(corrected_data, np.eye(4))
    if (plot == True):
        corrected_data = np.array(corrected_nifty.dataobj)
        plot_middle_slices(corrected_data)
    nib.save(corrected_nifty,store_dir)

def apply_affine_from_original_to_child(original, child, saveDir):
    child_nifty = nib.load(child)
    original_nifty = nib.load(original)
    original_nifty = nib.as_closest_canonical(original_nifty)
    child_nifty = nib.as_closest_canonical(child_nifty)
    child_data = np.array(child_nifty.dataobj)
    corrected = nib.Nifti1Image(child_data, original_nifty.affine)
    nib.save(corrected,saveDir)
def space_correction_folder(baseDir,saveDir):
    files = glob(baseDir + '/*.nii')
    
    for file in files:
        space_correction(file,str(saveDir + file[len(baseDir) :]))


In [ ]:
#--------------STEP six: check the size of our image files again step 5 can change the pixel size----------
# if this is true for some volumes you need to crop or downsample them again.. which will have negative impact on your
# rigid registration....
files = glob('.\\cropped\\downsampled\\64_rotation\\'+ '/*.nii')
    
for file in files:
    a,b = load_data(file)
    print(a.shape)